In [1]:
import pandas as pd
import numpy as np
import os
import simplejson
import json

In [2]:
def load_file(path):
    file_path = path
    df = pd.read_csv(file_path)
    #Automatic fixes
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df


In [3]:
def file_params(df):
    params = {}
    #size
    params['size'] = {}
    params['size']['rows'] = int(df.shape[0])
    params['size']['cols'] = int(df.shape[1])

    #missing values
    params['missing'] = {}
    ##rows
    params['missing']['rows'] = int(df[df.isna().any(axis=1)].shape[0])
    params['missing']['rowsPercent'] = float(round(params['missing']['rows'] / params['size']['rows'], 7) * 100)
    ##cells
    params['missing']['cells'] = int(df.isna().sum().sum())
    params['missing']['cellsPercent'] = float(round(params['missing']['cells'] / (params['size']['rows'] * params['size']['cols']), 7) * 100)
    ##columns
    missingColumn = df.isna().sum()
    params['missing']['cols'] = missingColumn[missingColumn != 0].to_dict()
    ###percent of total values in colums
    nanByColumnPercent = round(df.isna().sum() / df.count().sum() * 100, 4)
    params['missing']['colsPercent'] = nanByColumnPercent[nanByColumnPercent !=0].to_dict()
    ###percent ot total missing values
    nanColumnContributionPercent = round(df.isna().sum() / df.isna().sum().sum() * 100, 2)
    params['missing']['colsPercentContribution'] = nanColumnContributionPercent[nanColumnContributionPercent !=0].to_dict()

    #names
    params['names'] = {}
    params['names']['cols'] = list(df.columns.values)
    params['names']['colsReverse'] = list(df.columns.values)
    params['names']['colsReverse'].reverse()

    #describe
    params['describe'] = df.describe().to_dict()

    return params

In [4]:

target = 'NFLness'

df_sub1 = load_file('/Users/brandon/Desktop/test_data/NFL-Gen-Test_missing.csv')



df_sub2 = load_file('/Users/brandon/Desktop/test_data/NFL-Training_missing.csv')

df = pd.concat([df_sub1, df_sub2])

In [5]:
def fill_in_zeros(dict):
    if not 1 in dict:
        dict[1] = 0
    if not 0 in dict:
        dict[0] = 0
    return dict

def counts_to_percent(df):
    return round(df['counts'] / df['counts'].sum() * 100,1)


{'total': {'counts': {0: 246, 1: 242}, 'percent': {0: 50.4, 1: 49.6}}}

In [6]:

result = {}

#counts
total_df = pd.DataFrame(df[target].value_counts().astype(int))
total_df.rename(columns={target: 'counts'}, inplace=True)
#percentage
total_df['percent'] = counts_to_percent(total_df)


result['total'] = total_df.to_dict()



nan_df = df[df.isna().any(axis=1)][target].value_counts()
nan_df = fill_in_zeros(nan_df)

nan_df = pd.DataFrame(nan_df)
nan_df.rename(columns={target: 'counts'}, inplace=True)

nan_df['percent'] = round(nan_df['counts'] / total_df['counts'].sum() * 100,1)


result['nan'] = nan_df.to_dict()


non_nan_df = df[~df.isna().any(axis=1)][target].value_counts()
non_nan_df = fill_in_zeros(non_nan_df)

non_nan_df = pd.DataFrame(non_nan_df)
non_nan_df.rename(columns={target: 'counts'}, inplace=True)

non_nan_df['percent'] = counts_to_percent(non_nan_df)

non_nan_df['percent'] = round(non_nan_df['counts'] / total_df['counts'].sum() * 100,1)

result['non_nan'] = non_nan_df.to_dict()


major = df[target].value_counts().idxmax()
minor = df[target].value_counts().idxmin()

training_size = 25

test_major = result['total']['counts'][major] - training_size
test_minor = result['total']['counts'][minor] - training_size

total = test_major + test_minor + (2 * training_size)

partition = {
    'train': {
        'total': {
            'counts': training_size * 2,
            'percent': round(training_size * 2 / total * 100, 1)
        },
        'counts' : {
            major: training_size,
            minor: training_size
        },
        'percent': {
            major: round(training_size / total * 100, 1),
            minor: round(training_size / total * 100, 1),
        }
    },
    'test': {
        'total':  {
            'counts': test_major + test_minor,
            'percent': round((test_major + test_minor) / total * 100, 1)
        },
        'counts' : {
            major: test_major,
            minor: test_minor
        },
        'percent': {
            major: round(test_major / total * 100, 1),
            minor: round(test_minor / total * 100, 1),
        }

    }
}


#TODO deal with if equal

json = {
    'class': {
        'major': int(major),
        'minor': int(minor)
    },
    'describe': result
}

json





simplejson.dumps(json, ignore_nan=True)

'{"class": {"major": 0, "minor": 1}, "describe": {"total": {"counts": {"0": 246, "1": 242}, "percent": {"0": 50.4, "1": 49.6}}, "nan": {"counts": {"1": 17, "0": 0}, "percent": {"1": 3.5, "0": 0.0}}, "non_nan": {"counts": {"0": 246, "1": 225}, "percent": {"0": 50.4, "1": 46.1}}}}'

In [13]:
df.iloc[0:5, 0:5]]

SyntaxError: unmatched ']' (3213029144.py, line 1)